In [ ]:
from HelmholtzSolver2D import *
from ExampleBoundaries import square

c         = 344.0 # speed of sound [m/s]
rho       = 1.205 # density of air [kg/m^3]
frequency = 400.0 # frequency [Hz]
k         = frequency_to_wavenumber(frequency)

# Test Problem 1
# Dirichlet boundary condition with phi = sin(k/sqrt(2)*x) * sin(k/sqrt(2)*y)
#
solver = InteriorHelmholtzSolver2D(square())
boundary_condition = solver.dirichlet_boundary_condition()
boundary_condition.f[:] = np.sin(k/np.sqrt(2.0) * solver.centers[:,0]) \
                        * np.sin(k/np.sqrt(2.0) * solver.centers[:,1])

boundary_incidence = BoundaryIncidence(solver.len())
boundary_incidence.phi.fill(0.0)
boundary_incidence.v.fill(0.0)

interior_points = np.array([[0.0250, 0.0250],
                           [0.0750, 0.0250],
                           [0.0250, 0.0750],
                           [0.0750, 0.0750],
                           [0.0500, 0.0500]], dtype=np.float32)

interior_incident_phi = np.zeros(interior_points.shape[0], dtype=np.complex64)

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(interior_incident_phi, interior_points)
print("==============\n")
print(boundary_solution)
print(sample_solution)


# Test Problem 2
# von Neumann boundary condition such that phi = sin(k/sqrt(2) * x) * sin(k/sqrt(2) * y)
# Differentiate with respect to x and y to obtain outward normal:
# dPhi/dX = k/sqrt(2) * cos(k/sqrt(2) * x) * sin(k/sqrt(2) * y)
# dPhi/dY = k/sqrt(2) * sin(k/sqrt(2) * x) * cos(k/sqrt(2) * y)
boundary_condition = solver.neumann_boundary_condition()
w = k / np.sqrt(2.0)
for i in range(solver.centers.shape[0]):
    x = solver.centers[i, 0]
    y = solver.centers[i, 1]
    if (x < 1e-7):
        boundary_condition.f[i] = -w * np.cos(w * x) * np.sin(w * y)
    elif (x > 0.1 - 1e-7):
        boundary_condition.f[i] =  w * np.cos(w * x) * np.sin(w * y)
    elif (y < 1e-7):
        boundary_condition.f[i] = -w * np.sin(w * x) * np.cos(w * y)
    else:
        boundary_condition.f[i] =  w * np.sin(w * x) * np.cos(w * y)        

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(interior_incident_phi, interior_points)
print("\n\nTest Problem 2")
print("==============\n")
print(boundary_solution)
print(sample_solution)

   
# Test Problem 3
# The test problem computes the field produced by a unit source at
# the point (0.5,0.25) within the square with a rigid boundary.
# The rigid boundary implies the bondary condition v=0.
# The test problem computes the field produced by a unit source at
# the point (0.5,0.25) within the square with a rigid boundary.
# The incident velocity potential is given by {\phi}_inc=i*h0(kr)/4
# where r is the distance from the point (0.5,0.25)
boundary_condition = solver.neumann_boundary_condition()
boundary_condition.f.fill(0.0)

p = np.array([0.05, 0.025], dtype=np.float32)
for i in range(solver.centers.shape[0]):
    r = solver.centers[i] - p
    R = norm(r)
    boundary_incidence.phi[i] = 0.25j * hankel1(0, k * R)
    if solver.centers[i, 0] < 1e-7:
        boundary_incidence.v[i] = -0.25j * k * hankel1(1, k * R) * (-r[0] / R)
    elif solver.centers[i, 0] > 0.1 - 1e-7:
        boundary_incidence.v[i] = -0.25j * k * hankel1(1, k * R) * ( r[0] / R)
    elif solver.centers[i, 1] < 1e-7:
        boundary_incidence.v[i] = -0.25j * k * hankel1(1, k * R) * (-r[1] / R)
    elif solver.centers[i, 1] > 0.1 - 1e-7:
        boundary_incidence.v[i] = -0.25j * k * hankel1(1, k * R) * ( r[1] / R)
    else:
        assert False, "All cases must be handled above."
        
for i in range(interior_incident_phi.size):
    r = interior_points[i] - p
    R = norm(r)
    interior_incident_phi[i] = 0.25j * hankel1(0, k * R)
       
boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(interior_incident_phi, interior_points)
print("\n\nTest Problem 3")
print("==============\n")
print(boundary_solution)
print(sample_solution)


Copyright (C) 2017&ndash;20019 Frank Jargstorff

This file is part of the AcousticBEM library.

AcousticBEM is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

AcousticBEM is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with AcousticBEM. If not, see http://www.gnu.org/licenses/.